In [1]:
import findspark

In [3]:
findspark.init('/home/gerardo-rodriguez/spark-4.0.0-bin-hadoop3')

In [4]:
from pyspark.sql import SparkSession

In [5]:
spark = SparkSession.builder.appName('mylogreg').getOrCreate()

Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
25/08/19 10:36:12 WARN Utils: Your hostname, Lanz-Lenovo, resolves to a loopback address: 127.0.1.1; using 192.168.1.145 instead (on interface wlp2s0)
25/08/19 10:36:12 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/08/19 10:36:13 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [6]:
from pyspark.ml.classification import LogisticRegression

In [8]:
my_data = spark.read.format('libsvm').load('sample_libsvm_data.txt')

25/08/19 10:37:03 WARN LibSVMFileFormat: 'numFeatures' option not specified, determining the number of features by going though the input. If you know the number in advance, please specify it via 'numFeatures' option to avoid the extra scan.


In [10]:
my_data.show()

+-----+--------------------+
|label|            features|
+-----+--------------------+
|  0.0|(692,[127,128,129...|
|  1.0|(692,[158,159,160...|
|  1.0|(692,[124,125,126...|
|  1.0|(692,[152,153,154...|
|  1.0|(692,[151,152,153...|
|  0.0|(692,[129,130,131...|
|  1.0|(692,[158,159,160...|
|  1.0|(692,[99,100,101,...|
|  0.0|(692,[154,155,156...|
|  0.0|(692,[127,128,129...|
|  1.0|(692,[154,155,156...|
|  0.0|(692,[153,154,155...|
|  0.0|(692,[151,152,153...|
|  1.0|(692,[129,130,131...|
|  0.0|(692,[154,155,156...|
|  1.0|(692,[150,151,152...|
|  0.0|(692,[124,125,126...|
|  0.0|(692,[152,153,154...|
|  1.0|(692,[97,98,99,12...|
|  1.0|(692,[124,125,126...|
+-----+--------------------+
only showing top 20 rows


In [11]:
my_log_reg_model = LogisticRegression()

In [12]:
fitted_logreg_model = my_log_reg_model.fit(my_data)

In [13]:
log_summary = fitted_logreg_model.summary

In [15]:
log_summary.predictions.printSchema()

root
 |-- label: double (nullable = true)
 |-- features: vector (nullable = true)
 |-- rawPrediction: vector (nullable = true)
 |-- probability: vector (nullable = true)
 |-- prediction: double (nullable = false)



In [17]:
log_summary.predictions.show()

+-----+--------------------+--------------------+--------------------+----------+
|label|            features|       rawPrediction|         probability|prediction|
+-----+--------------------+--------------------+--------------------+----------+
|  0.0|(692,[127,128,129...|[20.3777627514875...|[0.99999999858729...|       0.0|
|  1.0|(692,[158,159,160...|[-21.114014198852...|[6.76550380011201...|       1.0|
|  1.0|(692,[124,125,126...|[-23.743613234684...|[4.87842678711891...|       1.0|
|  1.0|(692,[152,153,154...|[-19.192574012724...|[4.62137287296030...|       1.0|
|  1.0|(692,[151,152,153...|[-20.125398874706...|[1.81823629111716...|       1.0|
|  0.0|(692,[129,130,131...|[20.4890549504206...|[0.99999999873608...|       0.0|
|  1.0|(692,[158,159,160...|[-21.082940212796...|[6.97903542836027...|       1.0|
|  1.0|(692,[99,100,101,...|[-19.622713503561...|[3.00582577442810...|       1.0|
|  0.0|(692,[154,155,156...|[21.1594863606525...|[0.99999999935352...|       0.0|
|  0.0|(692,[127

In [18]:
train_data, test_data = my_data.randomSplit([0.7,0.3])

In [19]:
final_model = LogisticRegression()

In [20]:
fit_final_model = final_model.fit(train_data)

In [21]:
prediction_and_labels = fit_final_model.evaluate(test_data)

In [22]:
prediction_and_labels.predictions.show()

+-----+--------------------+--------------------+--------------------+----------+
|label|            features|       rawPrediction|         probability|prediction|
+-----+--------------------+--------------------+--------------------+----------+
|  0.0|(692,[95,96,97,12...|[21.0262608756310...|[0.99999999926139...|       0.0|
|  0.0|(692,[100,101,102...|[2.31473992398679...|[0.91009045808357...|       0.0|
|  0.0|(692,[122,123,124...|[18.6860158048317...|[0.99999999233051...|       0.0|
|  0.0|(692,[122,123,148...|[30.0963932985588...|[0.99999999999991...|       0.0|
|  0.0|(692,[124,125,126...|[37.8944403200781...|           [1.0,0.0]|       0.0|
|  0.0|(692,[124,125,126...|[22.5866299658270...|[0.99999999984485...|       0.0|
|  0.0|(692,[124,125,126...|[22.6182422968981...|[0.99999999984967...|       0.0|
|  0.0|(692,[126,127,128...|[44.2838476931232...|           [1.0,0.0]|       0.0|
|  0.0|(692,[128,129,130...|[28.5177450318809...|[0.99999999999958...|       0.0|
|  0.0|(692,[152

In [24]:
from pyspark.ml.evaluation import (BinaryClassificationEvaluator, 
                                    MulticlassClassificationEvaluator)

In [25]:
my_eval = BinaryClassificationEvaluator()

In [26]:
my_final_roc = my_eval.evaluate(prediction_and_labels.predictions)

In [27]:
my_final_roc

0.9930555555555555